In [1]:
import pandas as pd
import subprocess
from glob import glob
from datetime import datetime
import numpy as np
import re
from glob import glob
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import pearsonr
import matplotlib.pyplot as plt 
import gzip
import dask.dataframe as dd
from tqdm import tqdm
import gc
from collections import Counter
from sklearn.decomposition import PCA
from umap import UMAP
from pathlib import Path
import os
import inspect
from time import sleep
import sys
import itertools
from IPython.utils import io
import psycopg2
import warnings
import requests
from io import StringIO
from tqdm import tqdm_notebook
warnings.filterwarnings('ignore')
#conda create --name gpipe -c conda-forge openjdk=17 ipykernel pandas seaborn scikit-learn umap-learn psycopg2 dask
#conda activate gpipe
#conda install -c bioconda gcta plink snpeff
#wget https://snpeff.blob.core.windows.net/versions/snpEff_latest_core.zip



In [2]:
class vcfGeneratorExitools:
    def corrfunc(x, y, ax=None, **kws):
        r, _ = pearsonr(x, y)
        ax = ax or plt.gca()
        ax.annotate(f'ρ = {r:.2f}', xy=(.1, .9), xycoords=ax.transAxes)

    def get_vcf_header(vcf_path):
        with gzip.open(vcf_path, "rt") as ifile:
            for num, line in enumerate(ifile):
                if line.startswith("#CHROM"): return line.strip().split('\t')
                if num > 10000: return '-1'
        return '-1'

    def read_vcf(filename, method = 'pandas'):
        if method == 'dask':
            return dd.read_csv(filename,  compression='gzip', comment='#',  delim_whitespace=True, header=None, 
                               names = vcfGeneratorExitools.get_vcf_header(filename),blocksize=None,  dtype=str, ).repartition(npartitions = 100000)
        # usecols=['#CHROM', 'POS']
        return pd.read_csv(filename,  compression='gzip', comment='#',  delim_whitespace=True,
                           header=None, names = vcfGeneratorExitools.get_vcf_header(filename),  dtype=str )

    def name_gen2(filename):
        return filename.split('/')[-1].split('.')[0]
    
    def get_vcf_metadata(vcf_path):
        out = ''
        with gzip.open(vcf_path, "rt") as ifile:
            for num, line in enumerate(ifile):
                if line.startswith("#CHROM"): return out
                out += line 
        return '-1'

    def pandas2vcf(df, filename, metadata = ''):
        if  metadata == '':
            header = '\n'.join(["##fileformat=VCFv4.1",
            '##fileDate=20090805',
            '##source=myImputationProgramV3.1',
            '##reference=000GenomesPilot-NCBI36',
            '##INFO=<ID=NS,Number=1,Type=Integer,Description="Number of Samples With Data">',
            '##FORMAT=<ID=GT,Number=1,Type=String,Description="Genotype">',
            '##FORMAT=<ID=GQ,Number=1,Type=Integer,Description="Genotype Quality">',
            '##FORMAT=<ID=DP,Number=1,Type=Integer,Description="Read Depth">',
            '##FORMAT=<ID=HQ,Number=2,Type=Integer,Description="Haplotype Quality">']) + '\n'
        elif metadata[-4:] == '.vcf': header = get_vcf_metadata(metadata)

        with open(filename, 'w') as vcf: 
            vcf.write(header)
        df.to_csv(filename, sep="\t", mode='a', index=False)

    

def bash(call, verbose = 0, return_stdout = True, print_call = True):
    if print_call: print(call+'\n')
    out = subprocess.run(call.split(' '), capture_output = True) 
    if verbose and not return_stdout: print(out.stdout)
    
    if out.stderr: print(out.stderr.decode('ascii'))
    if return_stdout: return out.stdout.decode('ascii').strip().split('\n')
    return out

def qsub(call: str, queue = 'condo', walltime = 3, ppn = 1, out = 'log/', err = 'logerr/', project_dir = ''):
    err_path = f'{project_dir}{err}$PBS_JOBNAME.err'
    out_path = f'{project_dir}{out}$PBS_JOBNAME.out'
    call_path = f'{project_dir}{call}'
    return bash(f'qsub -q {queue} -l nodes=1:ppn={ppn} -j oe -o {out_path} -e {err_path} -l walltime={walltime}:00:00 {call_path}')
    

In [40]:
class gwas_pipe:
    def __init__(self, 
                 path: str = f'{Path().absolute()}/', 
                 use_tscc_modules: list = [],
                 all_genotypes: str = '/projects/ps-palmer/apurva/riptide/genotypes/round9_1',
                 gtca_path: str = '',
                 data: pd.DataFrame() = pd.DataFrame(),
                 traits: list = [],
                 project_name: str = 'test',
                 phewas_db: str = 'phewasdb.parquet.gz',
                 threads: int = 12):
        
        '''
        to run the gwas pipeline we need to ...
        
        '''
        
        if use_tscc_modules: bash(f'module load {" ".join(use_tscc_modules)}')
        self.gtca = 'gcta64' if not gtca_path else gtca_path
        self.path = path
        self.all_genotypes = all_genotypes
        df = data
        df.columns = df.columns.str.lower()
        self.df = df
        self.traits = [x.lower() for x in traits]
        self.phewas_db = phewas_db
        self.project_name = project_name
        
        self.sample_path = f'{self.path}genotypes/sample_rfids.txt'
        self.genotypes_subset = f'{self.path}genotypes/genotypes'
        self.genotypes_subset_vcf = f'{self.path}genotypes/genotypes_subset_vcf.vcf.gz'
        
        self.autoGRM = f'{self.path}grm/AllchrGRM'
        self.xGRM = f'{path}grm/xchrGRM'
        self.log = pd.DataFrame( columns = ['function', 'call', 'out'])
        self.thrflag = f'--thread-num {threads}'
        self.print_call = True
        self.chrList = ['x' if x == 21 else x for x in range(1,22)]
        
    def plink2Df(self, call, temp_out_filename = 'temp/temp', dtype = 'ld'):
        '''
        assigned THIAGO
        this function receives a plink call as a string, 
        runs the call, 
        then reads the output file with the ending dtype 
        and returns it as a pandas table
        '''
        
        full_call = re.sub(r' +', ' ', call + f'--out {self.path}{temp_out_filename}')
        
        ### add line to delete temp_out_filename before doing the 
        bash(full_call, print_call = False)

        try: out = pd.read_csv(f'{self.path}{temp_out_filename}.{dtype}', sep = '\s+')
        except:
            print(f"file not found")
            out = pd.DataFrame()
        return out 
    
    def plink(self, return_file = 'ld', outfile = '',  **kwargs):
        '''
        assigned THIAGO
        this function is a wrapper to run plink as a python function
        instead of having to have it as a string and bash call it
        if writing a flag that doesn't require a variable e.g.
        --make-grm use make_grm = ''
        if outfile is empty and return file is not
            we will return a pandas dataframe from the plink outputfile with ending return_file
        otherwise we will save the files under the filename outfile
        '''
        
        call = 'plink ' + ' '.join([f'--{k.replace("_", "-")} {v}'
                                    for k,v in kwargs.items()])
        if not outfile and return_file:
            return self.plink2Df(call ,dtype = f'{return_file}' ) 
        else:
            bash(call + f' --out {outfile}', print_call=False)
        return
        
    def bashLog(self, call, func, print_call = True):
        self.append2log(func, call , bash(re.sub(r' +', ' ', call), print_call = print_call))
        
        
    def append2log(self, func, call, out):
        self.log.loc[len(self.log)] = [func, call, out]
        with open(f'{self.path}/log/{func}.log', 'w') as f:
            f.write('\n'.join(out))
            
    def make_dir_structure(self, folders: list = ['data', 'genotypes', 'grm', 'log', 'logerr', 
                                            'results', 'temp', 'data/pheno', 'results/heritability', 
                                             'results/gwas',  'results/loco', 'results/qtls','results/eqtl',
                                                  'results/phewas', 'temp/r2']):
        
        '''
        assigned THIAGO
        this function will create the folder structure for the outputfiles of the project 
        
        '''
        for folder in folders:
            os.makedirs(f'{self.path}{folder}', exist_ok = True)
            
            
    def subsetSamplesFromAllGenotypes(self,samplelist: list = [], 
                                      use_rfid_from_df = True, sourceFormat = 'vcf', 
                                      geno: float = .1, maf: float = .005, hwe: float = 1e-10 ):
        
        '''
        this function will get the large round vcf (or plink) file, subset it based of the rfid from the dataframe
        (or a sample list if wanted), and filter based on missingness, minor allele frequency and hwe equilibrium
        '''
        
        
        funcName = inspect.getframeinfo(inspect.currentframe()).function
        
        os.makedirs(f'{self.path}genotypes', exist_ok = True)
        
        df = self.df[['rfid', 'rfid']] if use_rfid_from_df else pd.DataFrame([samplelist,samplelist]).T.astype(str)
        df.to_csv(f'{self.path}genotypes/sample_rfids.txt', index = False, header = None, sep = ' ')
        self.sample_names = samplelist
        #### add here a way so plink know which samples are male and which are female
        
        fmt_call = {'vcf': 'vcf', 'plink': 'bfile'}[sourceFormat]        
        
        extra_params = f'--geno {geno} --maf {maf} --hwe {hwe} --double-id --set-missing-var-ids @:#'
        
        self.bashLog(f'plink --{fmt_call} {self.all_genotypes} --keep {self.sample_path} {extra_params} {self.thrflag} --make-bed --out {self.genotypes_subset}',
                    funcName)
        
    def SubsampleMissMafHweFilter(self, sexfmt: str = 'M|F',  sexColumn: str = 'sex',  
                                  geno: float = .1, maf: float = .005, hwe: float = 1e-10,
                                  sourceFormat = 'vcf', remove_dup: bool = True, print_call: bool = False):
        
        '''
        more complex version of the function above that accounts for sample sex when subseting and performing the filters
        i.e. hwe is more complicated in chr X, chr Y and MT 
        '''
        
        funcName = inspect.getframeinfo(inspect.currentframe()).function
        
               
        fmt_call = {'vcf': 'vcf', 'plink': 'bfile'}[sourceFormat]  
        rmv = f' --double-id --set-missing-var-ids @:# ' if remove_dup else ' '
        sub = self.genotypes_subset
        
        mfList = sorted(sexfmt.split('|'))
        self.samples = {}
        for num, sx in enumerate(tqdm(mfList)): 
            
            dff = self.df[(self.df[sexColumn] == sx) & 
                          (self.df.astype(str).rfid.isin(vcftools.get_vcf_header(self.all_genotypes)))]
            dff[['rfid', 'rfid']].to_csv(f'{self.path}genotypes/sample_rfids_{sx}.txt', index = False, header = None, sep = ' ')
            self.samples[sx] = f'{self.path}genotypes/sample_rfids_{sx}.txt'
            
            filtering_flags = f' --geno {geno} --maf {maf} --hwe {hwe}'
            filtering_flags_justx = f''
            extra_flags = f'--not-chr X'  if num == 1 else ''
            
            self.bashLog(f'plink --{fmt_call} {self.all_genotypes} --keep {self.samples[sx]} {filtering_flags} {rmv} {extra_flags} {self.thrflag} --make-bed --out {sub}_{sx}',
                        f'{funcName}_subseting_{sx}')
            
            if num == 1:
                self.bashLog(f'plink --bfile {sub}_{sx} --chr x {self.thrflag} --make-bed --out {sub}_{sx}_xchr',
                        f'{funcName}_maleXsubset{sx}') #--out {sub}_{sx}_xchr
                male_1_x_filenames = [aa for aa in [f'{sub}_{sx}', f'{sub}_{sx}_xchr'] if len(glob(aa+'.*')) >= 5]
                male_gen_filenames = f'{self.path}/genotypes/temp_male_filenames'
                pd.DataFrame(male_1_x_filenames).to_csv(male_gen_filenames, index = False, header = None)
            else: female_hwe = f'{sub}_{sx}'
                
        print('merging sexes')        
        self.bashLog(f'plink --bfile {female_hwe} --merge-list {male_gen_filenames} {self.thrflag} {filtering_flags} --make-bed --out {sub}_hwe',
                        f'{funcName}_mergeSexes')
        
        self.genotypes_subset = f'{sub}_hwe'
        
    def generateGRM(self, autosome_list: list = list(range(1,21)), print_call: bool = False,
                    extra_chrs: list = ['xchr']):
        
        '''
        generates the grms, one per chromosome and one with all the chromossomes
        '''
        
        funcName = inspect.getframeinfo(inspect.currentframe()).function
                
        self.bashLog(f'{self.gtca} {self.thrflag} --bfile {self.genotypes_subset} --autosome-num 21 --autosome --make-grm-bin --out {self.autoGRM}',
            funcName, print_call = print_call)
        
        if 'xchr' in extra_chrs:
            self.bashLog(f'{self.gtca} {self.thrflag} --bfile {self.genotypes_subset} --make-grm-xchr --out {self.xGRM}',
                        f'{funcName}_chrX', print_call = print_call)
        
        for c in tqdm(autosome_list):
            self.bashLog(f'{self.gtca} {self.thrflag} --bfile {self.genotypes_subset} --chr {c} --make-grm-bin --out {self.path}grm/{c}chrGRM',
                        f'{funcName}_chr{c}',  print_call = print_call)

    def snpHeritability(self, print_call: bool = False):
        h2table = pd.DataFrame()
        for trait in tqdm(self.traits):
            trait_file = f'{self.path}data/pheno/{trait}.txt'
            out_file = f'{self.path}results/heritability/{trait}' 
            df.dropna(subset = ['rfid'])[['rfid', 'rfid', trait]].fillna('NA').astype(str).to_csv(trait_file, index = False, sep = ' ', header = None)
            
            self.bashLog(f'{self.gtca} --reml {self.thrflag} --pheno {trait_file} --grm {self.autoGRM} --out {out_file}',
                        f'snpHeritability_{trait}', print_call = print_call)
            
            a = pd.read_csv(f'{out_file}.hsq', skipfooter=6, sep = '\t',engine='python')
            b = pd.read_csv(f'{out_file}.hsq', skiprows=6, sep = '\t', header = None, index_col = 0).T.rename({1: trait})
            newrow = pd.concat(
                [a[['Source','Variance']].T[1:].rename({i:j for i,j in enumerate(a.Source)}, axis = 1).rename({'Variance': trait}),
                b],axis =1 )
            h2table= pd.concat([h2table,newrow])
            
        h2table.to_csv(f'{self.path}results/heritability/heritability.tsv', sep = '\t')
        return h2table
        
    def gwasPerChr(self, nchr: int = 21, print_call: bool = False, append_run_db: bool = True):
        for trait, chrom in tqdm(list(itertools.product(self.traits, range(1,nchr+1)))):
            if chrom == 21: chrom = 'x'
            self.bashLog(f'{self.gtca} {self.thrflag} --pheno {self.path}data/pheno/{trait}.txt --bfile {self.genotypes_subset} \
                                       --grm {self.path}grm/AllchrGRM   \
                                       --chr {chrom} \
                                       --mlma-subtract-grm {self.path}grm/{chrom}chrGRM  \
                                       --mlma --out {self.path}results/gwas/gwas_{chrom}_{trait}',
                        f'GWAS_{chrom}_{trait}', print_call = print_call)
    
    def GWAS(self, subtract_grm: bool = False, loco: bool = True , print_call: bool = False):
        #grm_flag = f'--grm {self.path}grm/AllchrGRM --mlma-subtract-grm {self.path}grm/AllchrGRM' if subtract_grm else ''
        grm_name = 'sub_grm' if subtract_grm else 'with_grm'
        loco_flag = '-loco' if loco else ''
        for trait in tqdm(self.traits):
            self.bashLog(f'{self.gtca} {self.thrflag} --pheno {self.path}data/pheno/{trait}.txt --bfile {self.genotypes_subset}\
                                       --mlma{loco_flag} --out {self.path}results/loco/{trait}',
                        f'GWAS_{grm_name}_{loco_flag[1:]}_{trait}',  print_call = print_call)
            
    
    
    def addGWASresultsToDb(self, researcher, round_version, gwas_version,filenames: list = [], pval_thresh = 1e-3, **kwards):
        
        
        if type(filenames) == str:
            filenames = [filenames]
            
        elif len(filenames) == 0 :
            filenames = sorted(glob(f'{self.path}results/loco/*.mlma'))
        all_new = []
        for file in tqdm(filenames):
            trait = re.findall('/([^/]+).mlma', file)[0].replace('.loco', '')
            new_info = pd.read_csv(file, sep = '\t').assign(**kwards).assign(uploadeddate = datetime.today().strftime('%Y-%m-%d'),
                                                                               researcher = researcher,
                                                                               project = self.project_name,
                                                                               trait = trait,
                                                                               filename = file,
                                                                               pval_threshold = pval_thresh,
                                                                               genotypes_file = self.all_genotypes,
                                                                               round_version = round_version,
                                                                               gwas_version = gwas_version)

            ###### add phewas version
            new_info['n_snps'] =  new_info.shape[0]
            # Subset by p-val 10e-2
            new_info = new_info.query(f'p < {pval_thresh}')
            all_new += [new_info]
            # Add more to subset
            # Test different keep values
        all_new = pd.concat(all_new)   
        
        try:
            alldata = pd.concat([all_new, pd.read_parquet(self.phewas_db)])
        except:
            print(f"Could not open phewas database in file: {self.phewas_db}, rebuilding db with only this project")
            alldata = all_new
        
        alldata.drop_duplicates(subset = ['researcher', 'project', 'round_version', 'trait', 'SNP', 'uploadeddate'], 
                                keep='first').to_parquet(self.phewas_db, index = False, compression='gzip')
        
        ###scp this to tscc
        
        
        
    def callQTLs(self, threshold: float = 5.3, window: int = 1e6, subterm: int = 2,
                 ldwin = 1e6, ldkb = 11000, ldr2 = .4, qtl_dist = 2*1e6, nchr: int = 21, NonStrictSearchDir = ''):
        thresh = 10**(-threshold)
        chr_list = ['x' if x == 21 else x for x in range(1,nchr+1)]
        
        if not NonStrictSearchDir:
            topSNPs = pd.DataFrame()
            for t, c in tqdm(list(itertools.product(self.traits, chr_list))):
                filename = f'{self.path}results/gwas/gwas_{c}_{t}.mlma'
                try:
                    topSNPs = pd.concat([topSNPs, pd.read_csv(filename, sep = '\t').query(f'p < {thresh}').assign(trait=t)])
                except:
                    print(f"didn't open {filename}, does it exist?")
        else:
            topSNPs = pd.concat([pd.read_csv(filename, sep = '\t').query(f'p < {thresh}').assign(trait=re.findall('/([^/]*).mlma', 
                                                                                                                  filename)[0].replace('.loco', '')) for
                                 filename in tqdm(glob(f"{NonStrictSearchDir}/*.mlma"))])

        out = pd.DataFrame()

        for (t, c), df in tqdm(topSNPs.groupby(['trait','Chr'])):
            df = df.set_index('bp')
            df.p = -np.log10(df.p)

            while df.query('p > @threshold').shape[0]:
                idx = df.p.idxmax()
                maxp = df.loc[idx]
                correlated_snps = df.loc[idx- window//2: idx + window//2].query('p > @maxp.p - @subterm')
                qtl = True if correlated_snps.shape[0] > 2 else False

                out = pd.concat([out,
                                 maxp.to_frame().T.assign(QTL= qtl)],
                                 axis = 0)

                ldfilename = f'{self.path}temp/r2/temp_qtl_n_{t}'
                self.bashLog(f'plink --bfile {self.genotypes_subset} --chr {c}  --ld-snp {maxp.SNP} \
                                     --ld-window {ldwin} {self.thrflag} \
                                     --nonfounders --r2  \
                                     --ld-window-r2 {ldr2} --out {ldfilename}',
                             f'qlt_{t}', False )#--ld_window_kb {ldkb}

                try: 
                    ldSNPS = pd.read_csv(f'{ldfilename}.ld', sep = r'\s+').SNP_B.to_list() + [maxp.SNP]
                    df = df.query('~(@idx - @qtl_dist//2 < index < @idx + @qtl_dist//2) and (SNP not in @ldSNPS)')
                except:
                    ldSNPS = [maxp.SNP]
                    df = df.query('(SNP not in @ldSNPS)') ##### should this be different than the one above?
                #if sum(cnt.values()) % 10 == 0: print(cnt)
                            

        out =  out.reset_index().rename({'index': 'bp'}, axis = 1).sort_values('trait')#.assign(project = self.project_name)
        self.allqtlspath = f'{self.path}/results/qtls/allQTLS.csv'
        out.to_csv(self.allqtlspath , index = False)
        return out.set_index('SNP')   
    
    def phewas(self, qtltable, ld_window = int(3e6), pval_threshold = 1e-5, nreturn: int =1,r2_threshold = .8,
              annotate = True, annotate_genome = 'rat6'):
        '''
        this function requires a QTL dataframe which is returned by callQTLs and a phewas database file
        we will
        
        '''
                
        db_vals = pd.read_parquet(self.phewas_db, compression='gzip').query(f'p < {pval_threshold} and project != "{self.project_name}"')        
        
        table_exact_match = db_vals.merge(qtltable.reset_index(), on = 'SNP', how = 'inner', suffixes = ('_phewasdb', '_QTL'))
        self.phewas_exact_match_path = f'{self.path}results/phewas/table_exact_match.csv'
        table_exact_match.to_csv(self.phewas_exact_match_path )
        #pd.concat([qtltable, db_vals] ,join = 'inner', axis = 1)
        
        nearby_snps = pd.concat([
             self.plink(bfile = self.genotypes_subset, chr = row.Chr, ld_snp = row.name,
               ld_window = ld_window, thread_num = 12, nonfounders = '', r2 = '')\
              .query(f'R2 > {r2_threshold}')\
              .drop(['CHR_A', 'BP_A', 'CHR_B'], axis = 1)\
              .rename({'SNP_A': 'SNP', 'SNP_B': 'NearbySNP', 'BP_B': 'NearbyBP'}, axis = 1)\
              .assign(**row.to_dict())\
              .set_index('SNP')
              for  _, row in tqdm(list(qtltable.iterrows())) ])
        
        table_window_match = db_vals.merge(nearby_snps.reset_index(), left_on= 'SNP', 
                                                         right_on='NearbySNP', how = 'inner', suffixes = ('_phewasdb', '_QTL'))
        
        
        self.phewas_window_r2 = f'{self.path}results/phewas/table_window_match_{ld_window}_{pval_threshold}_{r2_threshold}.csv'
        
        if table_window_match.shape[0] == 0:
            print('No QTL window matches')
            pd.DataFrame().to_csv(self.phewas_window_r2, index = False)
            return -1
            
        
        out = table_window_match.groupby([ 'SNP_QTL','project', 'trait_phewasdb'])\
                                .apply(lambda df : df[df.uploadeddate == df.uploadeddate.max()]\
                                                   .nsmallest(n = nreturn, columns = 'p_phewasdb'))\
                                                   .reset_index(drop = True)##.drop('index', axis = 1)
        
        out.to_csv(self.phewas_window_r2, index = False)
        if annotate:
            out = self.annotate(out.rename({'A1_phewasdb':'A1', 'A2_phewasdb': 'A2',
                                            'Chr_phewasdb':'Chr', 'bp_phewasdb':'bp'}, axis = 1), annotate_genome, 'NearbySNP', save = False)
        return out
        

        
        
    def eQTL(self, qtltable, pval_thresh: float = 1e-4, r2_thresh: float = .8, nreturn: int =1, ld_window: int = 3e6,
            tissue_list: list = ['Adipose', 'BLA','Brain','Eye','IL','LHb','Liver','NAcc','NAcc2','OFC','PL','PL2'],
            annotate = True, annotate_genome = 'rat6'):
        '''
        this function 
        
        '''
    
        out = []
        for tissue in tqdm(tissue_list,  position=0, desc="tissue", leave=True):

            tempdf = pd.read_csv(f'https://ratgtex.org/data/eqtl/{tissue}.cis_qtl_signif.txt.gz', sep = '\t').assign(tissue = tissue)\
                                                                                                             .rename({'variant_id': 'SNP'}, axis = 1)
            out += [pd.concat([
                   self.plink(bfile = self.genotypes_subset, chr = row.Chr, ld_snp = row.name,\
                   ld_window = ld_window, thread_num = 12, nonfounders = '', r2 = '')\
                  .drop(['CHR_A', 'BP_A', 'CHR_B'], axis = 1)\
                  .rename({'SNP_A': 'SNP', 'SNP_B': 'NearbySNP', 'BP_B': 'NearbyBP'}, axis = 1)\
                  .assign(**row.to_dict())\
                  .merge(tempdf, right_on= 'SNP',  left_on='NearbySNP', how = 'inner', suffixes = ('_QTL', '_eqtldb'))\
                  .query(f'R2 > {r2_thresh} and pval_nominal < {pval_thresh}')\
                  .nsmallest(nreturn, 'pval_nominal')
                  for  _, row in qtltable.iterrows() ])]

        out = pd.concat(out).reset_index(drop=True)
        if annotate:
            out = self.annotate(out, annotate_genome, 'NearbySNP', save = False)
        self.eqtl_path = f'{self.path}results/eqtl/eqtl.csv'
        out.to_csv(self.eqtl_path, index= False)
        
        return out

    def annotate(self, qtltable, genome: str = 'rat6', snpcol: str = 'SNP', save: bool = True):
        
        '''
        we will use the snpeff to query the annotations for the QTL results and for the phewas results
        '''
        d = {'rat6': 'Rnor_6.0.99', 'rat7':'mRatBN7.2.105'}[genome]
        #bash('java -jar snpEff/snpEff.jar download -v Rnor_6.0.99')
        #bash('java -jar snpEff/snpEff.jar download -v mRatBN7.2.105')    
        
        temp  = qtltable.reset_index()\
                        .loc[:,[ 'Chr', 'bp', snpcol, 'A1', 'A2']]\
                        .assign(QUAL = 40, FILTER = 'PASS' ,INFO = '', FORMAT = 'GT:GQ:DP:HQ')
        temp.columns = ["##CHROM","POS","ID","REF","ALT", 'QUAL', 'FILTER', 'INFO', 'FORMAT']
        temp['##CHROM'] = 'chr'+ temp['##CHROM'].astype(str)
        vcfGeneratorExitools.pandas2vcf(temp, f'{self.path}temp/test.vcf', metadata='')
        a = bash(f'java -Xmx8g -jar snpEff/snpEff.jar {d} -no-intergenic -no-intron -noStats {self.path}temp/test.vcf', 'snpefftest' )
        res =pd.read_csv(StringIO('\n'.join(a)),  comment='#',  delim_whitespace=True,  header=None, names = temp.columns,  dtype=str)  
        ann = res['INFO'].str.replace('ANN=', '').str.split('|',expand=True)
        ann.columns = ['alt_temp', 'annotation', 'putative_impact', 'gene', 'geneid', 'featuretype', 'featureid', 'transcriptbiotype',
                      'rank', 'HGVS.c', 'HGVS.p', 'cDNA_position|cDNA_len', 'CDS_position|CDS_len', 'Protein_position|Protein_len',
                      'distancetofeature', 'errors']
        ann.index = qtltable.index
        out = pd.concat([qtltable, ann], axis = 1).replace('', np.nan).dropna(how = 'all', axis = 1).drop('alt_temp', axis = 1)
        if save:
            self.annotatedtablepath = f'{self.path}results/qtls/finalqtl.csv'
            out.to_csv(self.annotatedtablepath, index= False)
        
        return out 
                

    def print_watermark():
        pass
    

            
            
            
            
    

In [41]:
df = pd.read_csv('~/Documents/GitHub/sanchest/hsrats_round9_1/Normalized_filtered_tom_jhou_U01_lowercase.csv')


In [42]:
gwas = gwas_pipe(path = 'test/',
                 all_genotypes = 'round9_1.vcf.gz',
                 data = df,
                 project_name = 'tj',
                 traits =  df.loc[:, 'locomotor1':].columns.tolist(),
                 threads=12,
                )


In [43]:
gwas.make_dir_structure()

In [ ]:
%time gwas.SubsampleMissMafHweFilter()

In [ ]:
%time gwas.generateGRM()

In [ ]:
%time gwas.snpHeritability()

In [ ]:
%time gwas.gwasPerChr()

In [ ]:
%time gwas.GWAS()

In [ ]:
%time gwas.addGWASresultsToDb(researcher='tsanches', project='tj', round_version='9.1', gwas_version='0.0.1')

In [15]:
#%time gwas.callQTLs()
ot = gwas.callQTLs(NonStrictSearchDir = 'test/results/loco/')
ot

100%|███████████████████████████████████████████████████████████████████████████████████| 10/10 [00:11<00:00,  1.10s/it]


,bp,Chr,A1,A2,Freq,b,se,p,trait,QTL
SNP,,,,,,,,,,
chr1:143001817,143001817,1,T,C,0.069241,-0.48236,0.103227,5.527049,delaypunishmentratio,False
chr9:110362547,110362547,9,T,C,0.211093,0.351662,0.073998,5.696644,locomotor1,True
chr9:111723693,111723693,9,C,T,0.199511,0.34787,0.074628,5.502892,locomotor1,True
chr5:19455786,19455786,5,G,A,0.333333,-0.251977,0.054721,5.384193,locomotor2,True
chr11:20205102,20205102,11,C,T,0.410714,0.247502,0.053392,5.448529,locomotor2,True
chr13:110174580,110174580,13,T,C,0.317942,-0.271674,0.059336,5.329553,progressiveratio,False
chr14:6165824,6165824,14,C,A,0.488064,0.245613,0.052523,5.534497,progressiveratio,True
chr2:72068315,72068315,2,C,T,0.147952,-0.370444,0.076975,5.826677,punishment,True
chr2:73124956,73124956,2,G,T,0.180387,-0.325543,0.068084,5.759393,punishment,True


In [ ]:
gwas.annotate(ot)

In [ ]:
gwas.eQTL(ot)

In [44]:
gwas.phewas(ot, annotate=True)

100%|███████████████████████████████████████████████████████████████████████████████████| 15/15 [00:07<00:00,  1.99it/s]


java -Xmx8g -jar snpEff/snpEff.jar Rnor_6.0.99 -no-intergenic -no-intron -noStats test/temp/test.vcf



,Chr,SNP_phewasdb,bp,A1,A2,Freq_phewasdb,b_phewasdb,se_phewasdb,p_phewasdb,uploadeddate,...,QTL,annotation,putative_impact,gene,geneid,featuretype,featureid,transcriptbiotype,HGVS.c,distancetofeature
0,11,chr11:20205102,20205102,C,T,0.410714,0.247502,0.053392,3.560170e-06,2023-01-12,...,True,None,None,None,None,None,None,None,None,None
1,13,chr13:110174580,110174580,T,C,0.317942,-0.271674,0.059336,4.682170e-06,2023-01-12,...,False,None,None,None,None,None,None,None,None,None
2,13,chr13:110174580,110174580,T,C,0.317942,0.273714,0.059547,4.294100e-06,2023-01-12,...,False,None,None,None,None,None,None,None,None,None
3,14,chr14:6165824,6165824,C,A,0.488064,0.245613,0.052523,2.920810e-06,2023-01-12,...,True,None,None,None,None,None,None,None,None,None
4,1,chr1:143001817,143001817,T,C,0.069241,-0.482360,0.103227,2.971330e-06,2023-01-12,...,False,None,None,None,None,None,None,None,None,None
5,2,chr2:72068315,72068315,C,T,0.147952,-0.370444,0.076975,1.490470e-06,2023-01-12,...,True,None,None,None,None,None,None,None,None,None
6,2,chr2:73124956,73124956,G,T,0.180387,-0.325543,0.068084,1.740230e-06,2023-01-12,...,True,None,None,None,None,None,None,None,None,None
7,2,chr2:75149524,75149524,G,A,0.182668,-0.318588,0.068973,3.855850e-06,2023-01-12,...,True,upstream_gene_variant,MODIFIER,U1,ENSRNOG00000052117,transcript,ENSRNOT00000086797.1,snRNA,n.-4593G>A,4593
8,3,chr3:37237803,37237803,G,A,0.352480,-0.266170,0.055719,1.779200e-06,2023-01-12,...,True,None,None,None,None,None,None,None,None,None
9,5,chr5:19455786,19455786,G,A,0.333333,-0.251977,0.054721,4.128640e-06,2023-01-12,...,True,None,None,None,None,None,None,None,None,None


In [ ]:
temp  = ot.reset_index().loc[:,[ 'Chr', 'bp', 'SNP', 'A1', 'A2']].assign(QUAL = 40, FILTER = 'PASS' ,INFO = '', FORMAT = 'GT:GQ:DP:HQ')
temp.columns = ["##CHROM","POS","ID","REF","ALT", 'QUAL', 'FILTER', 'INFO', 'FORMAT']
temp['##CHROM'] = 'chr'+ temp['##CHROM'].astype(str)
vcfGeneratorExitools.pandas2vcf(temp, f'{gwas.path}temp/test.vcf', metadata='' )
a = bash(f'java -Xmx8g -jar snpEff/snpEff.jar Rnor_6.0.99 -no-intergenic -no-intron -noStats {gwas.path}temp/test.vcf', 'snpefftest' )
res =pd.read_csv(StringIO('\n'.join(a)),  comment='#',  delim_whitespace=True,  header=None, names = temp.columns,  dtype=str)  
ann = res['INFO'].str.replace('ANN=', '').str.split('|',expand=True)
ann.columns = ['alt', 'annotation', 'putative_impact', 'gene', 'geneid', 'featuretype', 'featureid', 'transcriptbiotype',
              'rank', 'HGVS.c', 'HGVS.p', 'cDNA_position|cDNA_len', 'CDS_position|CDS_len', 'Protein_position|Protein_len',
              'distancetofeature', 'errors']
ann.index = ot.index
out = pd.concat([ot, ann], axis = 1)

In [11]:
1/10**5.3

5.011872336272724e-06

In [13]:
db_vals = pd.read_parquet(gwas.phewas_db).query(f'p < {1e-5} and project != "{gwas.project_name}"')

In [14]:
        nearby_snps = pd.concat([
             self.plink(bfile = self.genotypes_subset, chr = row.Chr, ld_snp = row.name,
               ld_window = ld_window, thread_num = 12, nonfounders = '', r2 = '')\
              .query(f'R2 > {r2_threshold}')\
              .drop(['CHR_A', 'BP_A', 'CHR_B'], axis = 1)\
              .rename({'SNP_A': 'SNP', 'SNP_B': 'NearbySNP', 'BP_B': 'NearbyBP'}, axis = 1)\
              .assign(**row.to_dict())\
              .set_index('SNP')
              for  _, row in tqdm(list(qtltable.iterrows())) ])
        
        table_window_match = db_vals.merge(nearby_snps.reset_index(), left_on= 'SNP', 
                                                         right_on='NearbySNP', how = 'inner', suffixes = ('_phewasdb', '_QTL'))

,Chr,SNP,bp,A1,A2,Freq,b,se,p,uploadeddate,researcher,project,trait,filename,genotypes_file,round_version,gwas_version,n_snps
7507695,1,chr1:142947467,142947467,A,G,0.074440,-0.443599,0.100037,0.000009,2023-01-12,tsanches,tomjhou2,delaypunishmentratio,test/results/loco/delaypunishmentratio.loco.mlma,round9_1.vcf.gz,9.1,0.0.1,4598918
7507697,1,chr1:142948896,142948896,T,C,0.074440,-0.443599,0.100037,0.000009,2023-01-12,tsanches,tomjhou2,delaypunishmentratio,test/results/loco/delaypunishmentratio.loco.mlma,round9_1.vcf.gz,9.1,0.0.1,4598918
7507698,1,chr1:142949913,142949913,T,C,0.074440,-0.443599,0.100037,0.000009,2023-01-12,tsanches,tomjhou2,delaypunishmentratio,test/results/loco/delaypunishmentratio.loco.mlma,round9_1.vcf.gz,9.1,0.0.1,4598918
7507700,1,chr1:142952269,142952269,T,C,0.074440,-0.443599,0.100037,0.000009,2023-01-12,tsanches,tomjhou2,delaypunishmentratio,test/results/loco/delaypunishmentratio.loco.mlma,round9_1.vcf.gz,9.1,0.0.1,4598918
7507701,1,chr1:142953106,142953106,T,C,0.074440,-0.443599,0.100037,0.000009,2023-01-12,tsanches,tomjhou2,delaypunishmentratio,test/results/loco/delaypunishmentratio.loco.mlma,round9_1.vcf.gz,9.1,0.0.1,4598918
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12030975,13,chr13:110188376,110188376,A,G,0.319317,0.264789,0.059338,0.000008,2023-01-12,tsanches,tomjhou2,punishmenteffortratio,test/results/loco/punishmenteffortratio.loco.mlma,round9_1.vcf.gz,9.1,0.0.1,4598918
12030976,13,chr13:110188437,110188437,T,C,0.319317,0.264789,0.059338,0.000008,2023-01-12,tsanches,tomjhou2,punishmenteffortratio,test/results/loco/punishmenteffortratio.loco.mlma,round9_1.vcf.gz,9.1,0.0.1,4598918
12031518,13,chr13:110681227,110681227,T,G,0.323963,0.265641,0.059408,0.000008,2023-01-12,tsanches,tomjhou2,punishmenteffortratio,test/results/loco/punishmenteffortratio.loco.mlma,round9_1.vcf.gz,9.1,0.0.1,4598918
12031867,13,chr13:110904823,110904823,T,C,0.313673,0.269657,0.060125,0.000007,2023-01-12,tsanches,tomjhou2,punishmenteffortratio,test/results/loco/punishmenteffortratio.loco.mlma,round9_1.vcf.gz,9.1,0.0.1,4598918


In [ ]:
test.groupby('project').apply(lambda df: df.query(f'uploadeddate == "{test.uploadeddate.max()}"'))

In [ ]:
test.nlargest(1, columns ='uploadeddate', keep = 'all')

In [ ]:
test

In [ ]:
out

In [ ]:
tissue_list = ['Adipose', 'BLA','Brain','Eye','IL','LHb','Liver','NAcc','NAcc2','OFC','PL','PL2' ]
alleqtls = pd.concat([pd.read_csv(f'https://ratgtex.org/data/eqtl/{tissue}.cis_qtl_signif.txt.gz', sep = '\t').assign(tissue = tissue)
          for tissue in  tqdm(tissue_list)  ])

In [ ]:





pval_thresh = 1e-4
r2_thresh = .8
nreturn =1 #int( 1e5)
tissue_list = ['Adipose', 'BLA','Brain','Eye','IL','LHb','Liver','NAcc','NAcc2','OFC','PL','PL2' ]

out = []
for tissue in tqdm(tissue_list[:2],  position=0, desc="tissue", leave=True):

    tempdf = pd.read_csv(f'https://ratgtex.org/data/eqtl/{tissue}.cis_qtl_signif.txt.gz', sep = '\t').assign(tissue = tissue)\
                                                                                                             .rename({'variant_id': 'SNP'}, axis = 1)
    out += [pd.concat([
         gwas.plink(bfile = gwas.genotypes_subset, chr = row.Chr, ld_snp = row.name,\
           ld_window = 1e6, thread_num = 12, nonfounders = '', r2 = '')\
          .drop(['CHR_A', 'BP_A', 'CHR_B'], axis = 1)\
          .rename({'SNP_A': 'SNP', 'SNP_B': 'NearbySNP', 'BP_B': 'NearbyBP'}, axis = 1)\
          .assign(**row.to_dict())\
          .merge(tempdf, right_on= 'SNP',  left_on='NearbySNP', how = 'inner', suffixes = ('_QTL', '_eqtldb'))
          .query(f'R2 > {r2_thresh} and pval_nominal < {pval_thresh}')\
          .nsmallest(nreturn, 'pval_nominal')
          for  _, row in tqdm(list(ot.iterrows()),  position=0, desc="qtl_row", leave=True) ])]

out = pd.concat(out).reset_index(drop=True)

In [ ]:
out

In [ ]:
for (_, row), tissue in list(product(ot.iterrows(), tissue_list)):
    display(row)
    print(tissue)

In [ ]:
table_window_match

In [ ]:
gwas.bashLog(f'java -Xmx8g -jar snpEff.jar', "tesi")

In [ ]:
!find /home/bonnie/miniconda3/envs/dask -name "snpEff.jar"

In [ ]:
# !conda install -c conda-forge snpeff -y
!java -jar snpEff.jar

In [ ]:
# java-jdk                  8.0.112                       1    bioconda
# snpeff                    5.1                  hdfd78af_0    bioconda
# !conda list

In [ ]:
!conda install -c bioconda java-jdk=8 -y

In [ ]:
!conda install -c conda-forge openjdk=8 -y
!conda install -c bioconda snpeff -y

In [ ]:
table_window_match = db_vals.reset_index().merge(nearby_snps.reset_index(), left_on= 'SNP', 
                                                         right_on='NearbySNP', how = 'inner', suffixes = ('_phewasdb', '_QTL'))

In [ ]:
table_window_match.R2

In [ ]:
db_vals = pd.read_parquet(gwas.phewas_db)

In [ ]:
import requests

In [ ]:


rg = gwas.plink(bfile = gwas.genotypes_subset, chr = testtest.Chr, ld_snp = testtest.name,
      ld_window = 1e6, thread_num = 12, nonfounders = '', r2 = '')\
      .drop(['CHR_A', 'BP_A', 'CHR_B'], axis = 1)\
      .rename({'SNP_A': 'SNP', 'SNP_B': 'NearbySNP', 'BP_B': 'NearbyBP'}, axis = 1)\
      .assign(**testtest.to_dict())\
      .set_index('SNP')

rg.query('R2 > .6')#['BP_B'].agg({"max":'max',"min": 'min'})

In [ ]:
def pull_data_from_db(project_name, savefiles = True):
    out = {}
    
    connection = psycopg2.connect(database = "PalmerLab_Datasets",
                              user = "postgres",
                              password = "palmerlab-amapostgres",
                              host = "palmerlab-main-database-c2021-08-02.c6sgfwysomht.us-west-2.rds.amazonaws.com",
                              port = '5432')
    cursor = connection.cursor()
    
    for key, val in {'wfu': f'{project_name}.wfu_master', 
                     'phenotypes': f'{project_name}.gwas_phenotypes', 
                     'tissue': f'sample_tracking.tissue' }.items():
        try:
            out[key] =  pd.read_sql(f"SELECT * FROM {val}", con = connection)
            if savefiles:
                os.makedirs(f'{project_name}', exist_ok = True)
                out[key].to_csv(f'{project_name}/db_{val}.csv', index = False)
        except:
            print(f'could not find {project_name}/{val}')
            out[key] = pd.DataFrame()

    cursor.close()
    connection.close()
    return out
    
    
    

In [ ]:
phewas_projects = ['p50_david_dietz_2020', 'p50_jerry_richards_2014', 'u01_suzanne_mitchell',
                   'p50_paul_meyer_2014', 'p50_paul_meyer_2020', 'r01_doug_adams',
                  'u01_olivier_george_cocaine', 'u01_olivier_george_oxycodone', 'u01_peter_kalivas_italy',
                  'u01_peter_kalivas_us', 'u01_tom_jhou', 'dean_baculum', 'p50_shelly_flagel_2014',
                   'lionikas_2014']

In [ ]:
for i in phewas_projects:
    pull_data_from_db(i)


In [ ]:
phewas_projects_avaliable = pd.DataFrame(sorted(glob('**/*.gwas_pheno*')), columns = ['phenotype_files'])
phewas_projects_avaliable['project'] = phewas_projects_avaliable.phenotype_files.apply(lambda x: x.split('/')[-2])

In [ ]:
phewas_projects_avaliable[['samplesize', 'phenotypesN']] = phewas_projects_avaliable.phenotype_files.apply(lambda x: list(pd.read_csv(x).shape)).tolist()
phewas_projects_avaliable = phewas_projects_avaliable.assign(genotypes_version = str(9.1), 
                                                             researcher = 'sanchest', 
                                                             gwasrundate = datetime.today().strftime('%Y-%m-%d'))

In [ ]:
phewas_projects_avaliable.to_csv('phewas_info_db.csv',)

In [ ]:
phewas_projects_avaliable

In [ ]:
for project in phewas_projects_avaliable.iterrows():
    gwas = 
    

In [ ]:
phewas_projects_avaliable

In [ ]:
df_rattacca = pd.read_csv('~/Documents/GitHub/sanchest/rattaca/CompleteG01.csv')

In [ ]:
gwas = gwas_pipe(path = 'rattacca/',
                 all_genotypes = 'round9_1.vcf.gz',
                 data = df,
                 traits =  df.loc[:, 'locomotor1':].columns.tolist(),
                 threads=12)

In [ ]:
rids with 3 zeros are gbs

In [ ]:

########################## download main vcf file round 9.1
#scp tsanches@tscc-login.sdsc.edu:/projects/ps-palmer/hs_rats/round9_1/Heterogenous-stock_n14780_10182022_QC_Sex_Het_pass_n13548.vcf.gz round9_1.vcf.gz
#scp tsanches@tscc-login.sdsc.edu:/projects/ps-palmer/hs_rats/round9_1/Heterogenous-stock_n14780_10182022_QC_Sex_Het_pass_n13548.vcf.gz.tbi round9_1.vcf.gz.tbi
#scp tsanches@tscc-login.sdsc.edu:/projects/ps-palmer/b3peng/gwas/p50_paul_meyer_2020/genotypes/dosages/dosages.txt round9_1_dosages.tsv
#scp tsanches@tscc-login.sdsc.edu:/projects/ps-palmer/tsanches/reference_panel/HOMCONC_FINAL_CHROMPOS.tsv round10CHRMPOS.tsv
#scp tsanches@tscc-login.sdsc.edu:/projects/ps-palmer/hs_rats/rattacafinalgenotypes/RattacaG01/RattacaG01_QC_Sex_Het_pass_n971.vcf.gz.tbi rattaca_genotypes.vcf.gz.tbi


        #print('geno maf filtering')
        #self.bashLog(f'plink --bfile {sub}_hwe --geno {geno} --maf {maf} --thread-num 12 --make-bed --out {sub}',
        #                f'{funcName}_geno_maf')
        
        ### LD pruning 
        #self.bashLog(f'plink --bfile {sub} --indep-pairwise 50 5 0.999 --thread-num 12 --out {sub}_pruned_50_5_999',
        #                f'{funcName}_findLDs')
        
        #self.bashLog(f'plink --bfile {sub}_hwe --geno {geno} --maf {maf} --thread-num 12 --make-bed --out {sub}_pruned',
        #                f'{funcName}_prune')